In [1]:
from torch.utils.data import DataLoader
import numpy as np
import matplotlib.pyplot as plt
import torch
from glob import glob
import seaborn as sns
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from tqdm import tqdm
import pickle
import random
import pandas as pd
import itertools
from scipy.stats import mannwhitneyu

import deep_snow.models
import deep_snow.dataset

In [2]:
# get paths to data
train_data_dir = '/mnt/Backups/gbrench/repos/deep-snow/data/subsets_v4/train'
train_path_list = glob(f'{train_data_dir}/ASO_50M_SD*.nc')

val_data_dir = '/mnt/Backups/gbrench/repos/deep-snow/data/subsets_v4/val'
val_path_list = glob(f'{val_data_dir}/ASO_50M_SD*.nc')

In [3]:
# define data to be returned by dataloader
selected_channels = [
    # ASO products
    'aso_sd', # ASO lidar snow depth (target dataset)
    'aso_gap_map', # gaps in ASO data
    
    # Sentinel-1 products
    'snowon_vv', # snow on Sentinel-1 VV polarization backscatter in dB, closest acquisition to ASO acquisition
    'snowon_vh', # snow on Sentinel-1 VH polarization backscatter in dB, closest acquisition to ASO acquisition
    'snowoff_vv', # snow off Sentinel-1 VV polarization backscatter in dB, closest acquisition to ASO acquisition
    'snowoff_vh', # snow off Sentinel-1 VH polarization backscatter in dB, closest acquisition to ASO acquisition
    'snowon_vv_mean', # snow on Sentinel-1 VV polarization backscatter in dB, mean of acquisition in 4 week period around ASO acquisition
    'snowon_vh_mean', # snow on Sentinel-1 VH polarization backscatter in dB, mean of acquisition in 4 week period around ASO acquisition
    'snowoff_vv_mean', # snow off Sentinel-1 VV polarization backscatter in dB, mean of acquisition in 4 week period around ASO acquisition
    'snowoff_vh_mean', # snow off Sentinel-1 VH polarization backscatter in dB, mean of acquisition in 4 week period around ASO acquisition
    'snowon_cr', # cross ratio, snowon_vh - snowon_vv
    'snowoff_cr', # cross ratio, snowoff_vh - snowoff_vv
    'delta_cr', # change in cross ratio, snowon_cr - snowoff_cr
    'rtc_gap_map', # gaps in Sentinel-1 data
    'rtc_mean_gap_map', # gaps in Sentinel-1 mean data
    
    # Sentinel-2 products 
    'aerosol_optical_thickness', # snow on Sentinel-2 aerosol optical thickness band 
    'coastal_aerosol', # snow on Sentinel-2 coastal aerosol band
    'blue', # snow on Sentinel-2 blue band
    'green', # snow on Sentinel-2 green band
    'red', # snow on Sentinel-2 red band
    'red_edge1', # snow on Sentinel-2 red edge 1 band
    'red_edge2', # snow on Sentinel-2 red edge 2 band
    'red_edge3', # snow on Sentinel-2 red edge 3 band
    'nir', # snow on Sentinel-2 near infrared band
    'water_vapor', # snow on Sentinel-2 water vapor
    'swir1', # snow on Sentinel-2 shortwave infrared band 1
    'swir2', # snow on Sentinel-2 shortwave infrared band 2
    'scene_class_map', # snow on Sentinel-2 scene classification product
    'water_vapor_product', # snow on Sentinel-2 water vapor product
    'ndvi', # Normalized Difference Vegetation Index from Sentinel-2
    'ndsi', # Normalized Difference Snow Index from Sentinel-2
    'ndwi', # Normalized Difference Water Index from Sentinel-2
    's2_gap_map', # gaps in Sentinel-2 data

    # PROBA-V global land cover dataset (Buchhorn et al., 2020)
    'fcf', # fractional forest cover
    
    # COP30 digital elevation model      
    'elevation',
    'slope',
    'aspect',
    'curvature',
    'tpi',
    'tri',

    # latitude and longitude
    'latitude',
    'longitude',

    # day of water year
    'dowy'
                    ]

In [4]:
def train_model(input_channels, epochs, n_layers):
    model = deep_snow.models.ResDepth(n_input_channels=len(input_channels), depth=n_layers)
    model.to('cuda');  # Run on GPU
    # Define optimizer and loss function
    optimizer = optim.Adam(model.parameters(), lr=0.0002)
    loss_fn = nn.MSELoss()
    epochs = epochs
    
    train_loss = []
    val_loss = []
    counter = 0
    
    # training and validation loop
    for epoch in range(epochs):
        print(f'\nStarting epoch {epoch+1}')
        epoch_loss = []
        val_temp_loss = []
    
        # Loop through training data with tqdm progress bar
        pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}", unit="batch", ncols=130)
        for data_tuple in pbar:
            model.train()
            optimizer.zero_grad()
    
            # read data into dictionary
            data_dict = {name: tensor for name, tensor in zip(selected_channels, data_tuple)}
            # prepare inputs by concatenating along channel dimension
            inputs = torch.cat([data_dict[channel] for channel in input_channels], dim=1).to('cuda')
    
            # generate prediction
            pred_sd = model(inputs)
    
            # Limit prediction to areas with valid data
            pred_sd = torch.where(data_dict['aso_gap_map'].to('cuda') + data_dict['rtc_gap_map'].to('cuda') + data_dict['s2_gap_map'].to('cuda') == 0, pred_sd, torch.zeros_like(pred_sd).to('cuda'))
            aso_sd = torch.where(data_dict['aso_gap_map'].to('cuda') + data_dict['rtc_gap_map'].to('cuda') + data_dict['s2_gap_map'].to('cuda') == 0, data_dict['aso_sd'].to('cuda'), torch.zeros_like(pred_sd).to('cuda'))
    
            # Calculate loss
            loss = loss_fn(pred_sd, aso_sd.to('cuda'))
            epoch_loss.append(loss.item())
    
            # Update tqdm progress bar with batch loss
            pbar.set_postfix({'batch loss': loss.item(), 'mean epoch loss': np.mean(epoch_loss)})
    
    
            loss.backward()  # Propagate the gradients in backward pass
            optimizer.step()
    
        train_loss.append(np.mean(epoch_loss))
        print(f'Training loss: {np.mean(epoch_loss)}')
    
        # Run model on validation data with tqdm progress bar
        for data_tuple in tqdm(val_loader, desc="Validation", unit="batch"):
            with torch.no_grad():
                model.eval()
                
                # read data into dictionary
                data_dict = {name: tensor for name, tensor in zip(selected_channels, data_tuple)}
                # prepare inputs by concatenating along channel dimension
                inputs = torch.cat([data_dict[channel] for channel in input_channels], dim=1).to('cuda')
        
                # generate prediction
                pred_sd = model(inputs)
        
                # Limit prediction to areas with valid data
                pred_sd = torch.where(data_dict['aso_gap_map'].to('cuda') + data_dict['rtc_gap_map'].to('cuda') + data_dict['s2_gap_map'].to('cuda') == 0, pred_sd, torch.zeros_like(pred_sd).to('cuda'))
                aso_sd = torch.where(data_dict['aso_gap_map'].to('cuda') + data_dict['rtc_gap_map'].to('cuda') + data_dict['s2_gap_map'].to('cuda') == 0, data_dict['aso_sd'].to('cuda'), torch.zeros_like(pred_sd).to('cuda'))
        
                # Calculate loss
                loss = loss_fn(pred_sd, aso_sd.to('cuda'))
                val_temp_loss.append(loss.item())

        # decay loss
        if epoch == 30:
            optimizer = optim.Adam(model.parameters(), lr=0.0001)
        if epoch == 60:
            optimizer = optim.Adam(model.parameters(), lr=0.00005)
        if epoch == 90:
            optimizer = optim.Adam(model.parameters(), lr=0.000025)

        # # calculate loss over previous 10 epochs for early stopping later
        # if epoch > 20:
        #     past_loss = np.median(val_loss[-20:-10])
    
        val_loss.append(np.mean(val_temp_loss))
        print(f'Validation loss: {np.mean(val_temp_loss)}')

        # # implement early stopping
        # if epoch > 20:
        #     current_loss = np.median(val_loss[-10:-1])
        #     if current_loss > past_loss:
        #         counter +=1
        #         if counter >= 10:
        #             print('early stopping triggered')
        #             break
        #     else:
        #         counter = 0

    return np.median(val_loss[-10:])

In [5]:
# define input channels for model
all_input_channels = [
    'blue',
    'green',
    'red',
    'nir',
    'swir1',
    'swir2',
    'ndvi',
    'ndsi',
    'ndwi',
    'fcf',
    'elevation',
    'slope',
    'aspect',
    'curvature',
    'tri',
    'tpi',
    'latitude',
    'longitude',
    'dowy',
    'snowon_vv',
    'snowon_vh',
    'snowoff_vv',
    'snowoff_vh',
    'snowon_cr',
    'snowoff_cr',
    'delta_cr',]

In [6]:
# core input channels
core_input_channels = ['delta_cr',
                       'ndsi',
                       'green',
                       'elevation'
                      ]

In [7]:
ancillary_input_channels = [channel for channel in all_input_channels if channel not in core_input_channels]

In [ ]:
n_epochs = 100
n_layers = 5

n_trials = 12
experiment_data = {}

for i in range(len(ancillary_input_channels)):
    print('---------------------------------------------------------')
    print(f'starting round {i}')

    experiment_loss = []
    
    if i == 0:
        print(f'round 1: establish baseline with core channels')
        for trial in range(n_trials):
            print(f'starting trial {trial+1}/{n_trials}')
            # subsample dataset
            train_path_subsample = random.sample(train_path_list, 256)
            val_path_subsample = random.sample(val_path_list, 256)
        
            # prepare training and validation dataloaders
            train_data = deep_snow.dataset.Dataset(train_path_subsample, selected_channels, norm=True)
            train_loader = torch.utils.data.DataLoader(dataset=train_data, batch_size=16, shuffle=True, num_workers=16)
            val_data = deep_snow.dataset.Dataset(val_path_subsample, selected_channels, norm=True)
            val_loader = torch.utils.data.DataLoader(dataset=val_data, batch_size=16, shuffle=True, num_workers=16)
            final_val_loss = train_model(core_input_channels, epochs=n_epochs, n_layers=n_layers)
            experiment_loss.append(final_val_loss)
            experiment_data['r0_baseline'] = experiment_loss
            # save to disk
            with open(f'../../loss/ResDepth_addition_v0.pkl', 'wb') as f:
                pickle.dump(experiment_data, f)
            baseline_loss = experiment_loss
            print(f'core channels baseline loss: {np.median(baseline_loss)}')
            round_channels = ancillary_input_channels
    else: 
        # new dictionary to store loss for the new round
        print(f'channels to add in round: {round_channels}')
        experiment_data[f'r{i}'] = {}
        next_round_channels = []
        # strop if only a single trial showed improvement in the last round
        if len(round_channels) < i:
            print('single experiment showed improved loss over last round, search completed')
            best_channels = min(experiment_data[f'r{i-1}'], key=lambda k: np.median(experiment_data[f'r{i-1}'][k]))
            print(f'final features to be added: {best_channels}')
            break
        else:
            # get combinations of number i of feature targeted for ablation
            channel_combinations = list(itertools.combinations(round_channels, i))
            for experiment, experiment_channels in enumerate(channel_combinations):
                experiment_loss = []
                print(f'starting experiment {experiment+1}/{len(channel_combinations)}, channels to add: {experiment_channels}')
                # select features to be added
                input_channels = core_input_channels + list(experiment_channels)
                for trial in range(n_trials):
                    # subsample dataset
                    train_path_subsample = random.sample(train_path_list, 256)
                    val_path_subsample = random.sample(val_path_list, 256)
                
                    # prepare training and validation dataloaders
                    train_data = deep_snow.dataset.Dataset(train_path_subsample, selected_channels, norm=True)
                    train_loader = torch.utils.data.DataLoader(dataset=train_data, batch_size=16, shuffle=True, num_workers=16)
                    val_data = deep_snow.dataset.Dataset(val_path_subsample, selected_channels, norm=True)
                    val_loader = torch.utils.data.DataLoader(dataset=val_data, batch_size=16, shuffle=True, num_workers=16)
                    print(f'starting trial {trial+1}/{n_trials}, channels to add: {experiment_channels}')
                    final_val_loss = train_model(input_channels, epochs=n_epochs, n_layers=n_layers)
                    experiment_loss.append(final_val_loss)
                    experiment_data[f'r{i}'][experiment_channels] = experiment_loss
                    # save to disk
                    with open(f'../../loss/ResDepth_addition_v0.pkl', 'wb') as f:
                        pickle.dump(experiment_data, f)
                    print(f'{experiment_channels} trial {trial+1} final val loss: {final_val_loss}')
                
                print(f'{experiment_channels} experiment loss: {np.median(experiment_loss)}, R0 baseline loss: {np.median(experiment_data["r0_baseline"])}')
                
                # update for next round if loss improves significantly
                if np.median(experiment_loss) < np.median(baseline_loss):
                    stat_val, p_val = mannwhitneyu(experiment_loss, baseline_loss)
                    if p_val < 0.5: # 50 percent chance improvement is not random
                        for channel in experiment_channels:
                            if channel not in next_round_channels:
                                next_round_channels.append(channel)

            # break if no combinations show improved loss over last round
            if len(next_round_channels) == 0:
                print('no improved loss over last round, search completed')
                best_channels = min(experiment_data[f'r{i-1}'], key=lambda k: np.median(experiment_data[f'r{i-1}'][k]))
                print(f'final features to be added: {best_channels}')
                break
            else:
                # update channels to be added
                round_channels = next_round_channels
                # find new baseline loss
                best_channels = min(experiment_data[f'r{i}'], key=lambda k: np.median(experiment_data[f'r{i}'][k]))
                baseline_loss = experiment_data[f'r{i}'][best_channels]
                print(f'new baseline: {best_channels}, val loss: {np.median(baseline_loss)}')

---------------------------------------------------------
starting round 0
round 1: establish baseline with core channels
starting trial 1/12

Starting epoch 1


Epoch 1/100: 100%|███████████████████████████████| 16/16 [00:06<00:00,  2.34batch/s, batch loss=0.000887, mean epoch loss=0.00304]


Training loss: 0.003041308231331641


Validation: 100%|████████████████████████████████████████████████████████████████████| 16/16 [00:05<00:00,  2.91batch/s]


Validation loss: 0.001871173779363744

Starting epoch 2


...

Epoch 6/100: 100%|████████████████████████████████| 16/16 [00:05<00:00,  2.75batch/s, batch loss=0.00254, mean epoch loss=0.00114]


Training loss: 0.0011366142516635591


Validation: 100%|████████████████████████████████████████████████████████████████████| 16/16 [00:05<00:00,  2.87batch/s]


Validation loss: 0.0010090968244185206

Starting epoch 7


Epoch 7/100:   0%|                                                                                      | 0/16 [00:00<?, ?batch/s]